# Set-up and Authorization

In [51]:
import sys
import os
import spotipy
import spotipy.util as util

os.environ["SPOTIPY_CLIENT_ID"]='b0b5be645e864041a2fad5d7ccdf0531'
os.environ["SPOTIPY_CLIENT_SECRET"]='86078e2a70f0476e9ec2785ed822b05e'
os.environ["SPOTIPY_REDIRECT_URI"]='http://localhost:8080/'

scope = 'user-library-read'

token = util.prompt_for_user_token('sxwang@uvic.ca', scope)
spotify = spotipy.Spotify(auth=token)

# Data Gathering

### Get featured playlist tracks and their albums

In [39]:
featured_playlists = spotify.featured_playlists(locale=None, country="CA", timestamp=None, limit=50, offset=0)

featured_track_ids = set()
album_ids = set()
for playlist in featured_playlists["playlists"]["items"]:
    for track in spotify.user_playlist("spotify", playlist['id'], fields="tracks, next")["tracks"]["items"]:
        featured_track_ids.add(track["track"]["id"])
        album_ids.add(track["track"]["album"]["id"])

### Get tracks from albums

In [40]:
album_track_ids = set()
for album in album_ids:
    for track in spotify.album_tracks(album)["items"]:
        album_track_ids.add(track["id"])

In [41]:
# DEBUG
print(len(featured_track_ids))
print(len(album_ids))
print(len(album_track_ids))

1352
1263
5544


# Data Pre-processing

### Get Spotify audio features for each track

In [59]:
attribs = ["danceability",
            "energy",
            "key",
            "loudness",
            "mode",
            "speechiness",
            "acousticness",
            "instrumentalness",
            "liveness",
            "valence",
            "tempo",
            "duration_ms",
            "time_signature",
            "featured",
          ]

def get_audio_data(track_ids, class_value):
    tracks_data = {}
    
    for id in track_ids:
        audio_data = spotify.audio_features(id)[0]
        if audio_data is None:
            print("id: ", id) # DEBUG, change to 'if audio_data is not None:'
        else:
            audio_data["featured"] = class_value
            tracks_data[id] = {k:v for (k,v) in audio_data.items() if k in attribs}
    
    return tracks_data

In [52]:
featured_tracks = get_audio_data(featured_track_ids, 1)

id:  6NGIfK4brC3Xc4l191RnjR


In [60]:
album_tracks = get_audio_data(album_track_ids, 0)

id:  6NGIfK4brC3Xc4l191RnjR


In [61]:
# DEBUG
print(len(featured_tracks))
print(len(album_tracks))

1351
5543


### Combine track databases such that featured track class values are not overwritten

In [67]:
db = featured_tracks
for k, v in album_tracks.items():
    if k not in db:
        db[k] = v
        
# DEBUG
print(len(db))

5543


### Export data

In [65]:
import csv

with open('data.csv', 'w') as f:
    writer = csv.DictWriter(f, fieldnames = attribs)

    writer.writeheader()
    for track in db.values():
        writer.writerow(track)

# Notes:

- Remove DEBUG prints
- May want to check for repeats between featured_track_ids and album_track_ids before getting audio features to save some processing time
- Consider adding log-in info to set-up code block so they can rerun our code
- Consider saving track IDs in Weka somehow for results comparison and accuracy-checking